### 채널별 전환 퍼널(Conversion Funnel) 시각화

In [27]:
query = """
with 
temp_act_0 as ( 
select sess_id, hit_type, action_type, channel_grouping
from ga.temp_funnel_base a
where a.action_type = '0'
), 
temp_hit_02 as ( 
select a.sess_id as home_sess_id, a.channel_grouping as home_cgrp
	, b.sess_id as plist_sess_id, b.channel_grouping as plist_cgrp
	, c.sess_id as pdetail_sess_id, c.channel_grouping as pdetail_cgrp
	, d.sess_id as cart_sess_id, d.channel_grouping as cart_cgrp
	, e.sess_id as purchase_sess_id, e.channel_grouping as pur_cgrp
from temp_act_0 a
	left join ga.temp_funnel_base b on (a.sess_id = b.sess_id and b.action_type = '1')
	left join ga.temp_funnel_base c on (b.sess_id = c.sess_id and c.action_type = '2')
	left join ga.temp_funnel_base d on (c.sess_id = d.sess_id and d.action_type = '3')
	left join ga.temp_funnel_base e on (d.sess_id = e.sess_id and e.action_type = '6')
)
select home_cgrp
 	, count(home_sess_id) as home_sess_cnt
	, count(plist_sess_id) as plist_sess_cnt
	, count(pdetail_sess_id) as pdetail_sess_cnt
	, count(cart_sess_id) as cart_sess_cnt
	, count(purchase_sess_id) as purchase_sess_cnt
from temp_hit_02
group by home_cgrp
"""
df = pd.read_sql_query(sql=query, con=postgres_engine)
df.head(10)

,home_cgrp,home_sess_cnt,plist_sess_cnt,pdetail_sess_cnt,cart_sess_cnt,purchase_sess_cnt
0,Affiliates,317,38,38,15,0
1,Direct,2232,377,375,129,31
2,Display,179,39,39,19,3
3,Organic Search,6892,1242,1241,373,48
4,Paid Search,445,98,98,30,9
5,Referral,1740,481,480,204,63
6,Social,14464,75,73,13,0


In [30]:
df.set_index('home_cgrp')

,home_sess_cnt,plist_sess_cnt,pdetail_sess_cnt,cart_sess_cnt,purchase_sess_cnt
home_cgrp,,,,,
Affiliates,317,38,38,15,0
Direct,2232,377,375,129,31
Display,179,39,39,19,3
Organic Search,6892,1242,1241,373,48
Paid Search,445,98,98,30,9
Referral,1740,481,480,204,63
Social,14464,75,73,13,0


In [31]:
df_t = df.set_index('home_cgrp').transpose()
df_t

home_cgrp,Affiliates,Direct,Display,Organic Search,Paid Search,Referral,Social
home_sess_cnt,317,2232,179,6892,445,1740,14464
plist_sess_cnt,38,377,39,1242,98,481,75
pdetail_sess_cnt,38,375,39,1241,98,480,73
cart_sess_cnt,15,129,19,373,30,204,13
purchase_sess_cnt,0,31,3,48,9,63,0


In [33]:
from plotly import graph_objects as go

fig = go.Figure()


fig.add_trace(go.Funnel(
    name = 'Direct',
    orientation = "h",
    y = ["home", "product list", "product_detail", "cart", "purchase"],
    x = df_t['Direct'],
    textposition = "inside",
    textinfo = "value+percent previous"))


fig.add_trace(go.Funnel(
    name = 'Organic Search',
    orientation = "h",
    y = ["home", "product list", "product_detail", "cart", "purchase"],
    x = df_t['Organic Search'],
    textposition = "inside",
    textinfo = "value+percent previous"))
'''
fig.add_trace(go.Funnel(
    name = 'Paid Search',
    orientation = "h",
    y = ["home", "product list", "product_detail", "cart", "purchase"],
    x = df_t['Paid Search'],
    textposition = "inside",
    textinfo = "value+percent previous"))
'''
fig.add_trace(go.Funnel(
    name = 'Referral',
    orientation = "h",
    y = ["home", "product list", "product_detail", "cart", "purchase"],
    x = df_t['Referral'],
    textposition = "inside",
    textinfo = "value+percent previous"))
'''
fig.add_trace(go.Funnel(
    name = 'Social',
    orientation = "h",
    y = ["home", "product list", "product_detail", "cart", "purchase"],
    x = df_t['Social'],
    textposition = "inside",
    textinfo = "value+percent previous"))
'''
fig.show()

### 개별 채널별로 subplot으로  Funnel 시각화

In [34]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Direct", "Organic Search", "Referral", "Social"))

fig.add_trace(go.Funnel(
    name = 'Direct',
    orientation = "h",
    y = ["home", "product list", "product_detail", "cart", "purchase"],
    x = df_t['Direct'],
    textposition = "inside",
    textinfo = "value+percent previous"), row=1, col=1)


fig.add_trace(go.Funnel(
    name = 'Organic Search',
    orientation = "h",
    y = ["home", "product list", "product_detail", "cart", "purchase"],
    x = df_t['Organic Search'],
    textposition = "inside",
    textinfo = "value+percent previous"), row=1, col=2)

fig.add_trace(go.Funnel(
    name = 'Referral',
    orientation = "h",
    y = ["home", "product list", "product_detail", "cart", "purchase"],
    x = df_t['Referral'],
    textposition = "inside",
    textinfo = "value+percent previous"), row=2, col=1)

fig.add_trace(go.Funnel(
    name = 'Social',
    orientation = "h",
    y = ["home", "product list", "product_detail", "cart", "purchase"],
    x = df_t['Social'],
    textposition = "inside",
    textinfo = "value+percent previous"), row=2, col=2)


fig.update_layout(height=800, width=800,
                  title_text="채널별 퍼널 전환")

fig.show()